In [ ]:
#group6, basic prep

%matplotlib inline
import matplotlib
import os              # This provides several system utilities
import pandas as pd    # This is the workhorse of data munging in Python
import numpy as np     # This is for general numerical operations 
import seaborn as sns  # This allows us to efficiently and beautifully plot
import networkx as nx  
import osmnx as ox
import statsmodels.api as sm
import matplotlib.pyplot as plt

# display settings (optional)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [1]:
#for the import process, next step is to create a string variable containing the directory of the file and a file name variable
infodir = r'C:\Users\EPRO\OneDrive - Wageningen University & Research\Course Channel Metropolitan Challenges 2025 - Paper Group 6 Parking space vs Food Space'
fname = infodir + r'\data\FILE_TO_REPLACE.csv'

g6h01 = pd.read_csv(fname)
#this is to verify
g6h01.head()

NameError: name 'pd' is not defined

In [ ]:
#Group6 dropping some rows according to some criteria
g6h01_1 = g6h01[g6h01['REPLACE0'] == 'COMMON VALUE']
#Rearrange some columns for easier reading
g6h01_1 = g6h01_1[['REPLACE', 'REPLACE', 'REPLACE', 'REPLACE'] + [col for col in g6h01_1.columns if col not in ['OBJECTNUMMER', 'Wijken', 'BRT_code', 'BRT_name']]]
g6h01_1.head()

In [ ]:
#group6 is purging according to some criteria
g6h01_2 = g6h01_1[(g6h01_1['REPLACE1'] <= 150) & (g6h01_1['REPLACE2'] <= 150)]
#we had prepared the db to be ordered in this point, but after getting the second database,
#it will be better to do that after the merger
## gpdf01_2 = gpdf01_2.sort_values(by=["Netto_PKD", "Bruto_PKD"], ascending=[False, True])
g6h01_2.head(10)

In [ ]:
#sorting the charts ascending

In [ ]:
#sortinf the charts descending

In [ ]:
#area for plotting some graphs

In [ ]:
#perhaps a map here

In [ ]:
#prepping OLS with one regressor
# 1) Add constant
X = g6h01_2.add_constant(g6h01_2[['indvr1']])
y = g6h01_2['dpnvr']

# 2) Fit model
model_simple = sm.OLS(y, X).fit()

# 3) Inspect results
print(model_simple.summary())

In [ ]:
#preparing the analysis with multiple variables
features = ['indvr1', 'indvr2', 'indvr3', 'indvr4', 'indvr5', 'indvr6']
X_multi = sm.add_constant(g6h01_2[features])
y = g6h01_2['dpnvr']

model_multi = sm.OLS(y, X_multi).fit()
print(model_multi.summary())


In [ ]:
#adding the quadratic term
g6h01_2[''] = g6h01_2['']**2
features_quad = ['', '', '', '', '', '', '']
X_quad = sm.add_constant(g6h01_2[features_quad])
model_quad = sm.OLS(y, X_quad).fit()
print(model_quad.summary())

# Example: marginal effect of temperature at 10°C is beta_tmax + 2*beta_tmax2*10
beta_t = model_quad.params['']
beta_t2 = model_quad.params['']
me_10 = beta_t + 2*beta_t2*10
me_20 = beta_t + 2*beta_t2*20
print(f'Marginal effect of something1: {me_10:.2f} dpnvr')
print(f'Marginal effect of something2: {me_20:.2f} dpnvr')

In [ ]:
# Observed vs Predicted
y_pred = model_quad.fittedvalues

plt.figure()
plt.scatter(y_pred, y)
plt.xlabel('Future green area if intervined')
plt.ylabel('Current green area')
plt.title('Current vs Future (quadratic model)')
plt.show()

# Residuals vs Fitted
resid = model_quad.resid
plt.figure()
plt.scatter(y_pred, resid)
plt.axhline(0)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted (quadratic model)')
plt.show()

print('R^2:', model_quad.rsquared)
print('Adj. R^2:', model_quad.rsquared_adj)
print('RMSE:', np.sqrt((resid**2).mean()))
